In [1]:
from itertools import groupby
import tensorflow as tf
import random
import torch
import numpy as np
####### READING THE FASTA CHAIN DATA FROM THE FILE ############

def fasta_iter(name):
    fh = open(name)

    faiter = (x[1] for x in groupby(fh, lambda line: line[0] == ">"))

    for header in faiter:
        # drop the ">"
        headerStr = header.__next__()[1:].strip()

        # join all sequence lines to one.
        seq = "".join(s.strip() for s in faiter.__next__())

        yield (headerStr, seq)

data=fasta_iter("Ecoli_prom.fa.txt")
training_data=[]

######## CONVERTING DNA SEQUENCE INTO ONE HOT ENCODING ##########

for chain in data:
    headerStr, seq = chain
    #print(seq)
    temp=[]
    for i in seq:
        if(i=='A'):
            temp.append(1)
        if(i=='T'):
            temp.append(2)
        if(i=='G'):
            temp.append(3)
        if(i=='C'):
            temp.append(4)
    
    training_data.append(temp)
       
    
    
data=fasta_iter("Ecoli_non_prom.fa.txt")
for chain in data:
    headerStr, seq = chain
    temp=[]
    for i in seq:
        if(i=='A'):
            temp.append(1)
        if(i=='T'):
            temp.append(2)
        if(i=='G'):
            temp.append(3)
        if(i=='C'):
            temp.append(4)
        
    training_data.append(temp)

#training_data

In [2]:
########### CREATING LABELS FOR THE OUTPUT ###########
labels=[]
for i in range(839):
    labels.append(1)
for i in range(3000):
    labels.append(0)

#### SHUFFLE THE DATA ########    

comb=list(zip(training_data,labels))
random.shuffle(comb)
training_data,labels=zip(*comb)


print(np.shape(training_data))
print(np.shape(labels))

training_data=np.reshape(training_data,(3839,81))

#print(np.shape(training_data))


(3839, 81)
(3839,)


In [3]:
####### SPLITTING DATA INTO TRAIN,VALIDATION AND TEST ############
X_train=training_data[:3300]
Y_train=labels[:3300]

X_test=training_data[3300:]
Y_test=labels[3300:]

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

/home/vinayak/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [5]:
parameters = {'bootstrap': True,
              'min_samples_leaf': 5,
              'n_estimators': 300, 
              'min_samples_split': 10,
              'max_features': None,
              'max_depth': 20,
              'max_leaf_nodes': None,
              'warm_start' : True
             }

In [6]:
RF_model = RandomForestClassifier(**parameters)
RF_model.fit(X_train, Y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0, warm_start=True)

In [8]:
RF_predictions = RF_model.predict(X_test)
score = accuracy_score(Y_test ,RF_predictions)
print("Testing accuracy:", score*100)
RF_predictions = RF_model.predict(X_train)
score = accuracy_score(Y_train ,RF_predictions)
print("Training accuracy:", score*100)

Testing accuracy: 89.42486085343229
Training accuracy: 97.51515151515152
